# Polynomials
Object-oriented methods are useful for presenting many items, including polynomial functions.

## Goal
We want to represent polynomials in Python and implement some basic operations between polynomials, such as addition, subtraction, multiplication, and division.

## How do we want to represent polynomials?
Suppose we have a polynomial of the form:

$a_0 + a_1x + a_2x^2 + ... a_nx^n$

where $a_0, a_1, ..., a_n$ are the coefficients of the polynomial. For simplicity, we will only consider real polynomials.

### Subclassing Lists
We can create an entirely new object from scratch to represent polynomials, or we can subclass an existing Python container. For one, lists appear to be a reasonable way to represent polynomials if we store the nth term in the nth place in the list. For example, 
$x^2 + 3x + 10$ can be represented using the list `[10, 3, 1]`

In [1]:
class Polynomial(list):
    def __init__(self, *args):
        # Put arguments into a list, and pass them to list's __init__()
        super(Polynomial, self).__init__(list(args))

Calling the `super()` function under `__init__` gives our `Polynomial` class the same properties as its parent class `list`. Furthermore, the `*args` argument allows us to take in an arbitrary number of arguments during initalization. In our case, these arguments will be the coefficients of our polynomial. We then pass these arguments to the parent `list` class's `__init__` function, which creates our list.

As a result, we can initalize polynomials like this:
`a = Polynomial(`$a_0$, $a_1$, $a_2$, ..., $a_n$`)`


In [2]:
x = Polynomial(1, 2, 3)
x

[1, 2, 3]

## Making Polynomials Pretty
Of course, we are not accustomed to seeing polynomials like `[1, 2, 3]`. Rather, we prefer something like `1 + 2x + 3x^2`. We can make this happen by overriding the list's default `__repr__()` method, which controls how our objects are represented. The output of `__repr__()` for any object should be a string.

In [3]:
class Polynomial(list):
    def __init__(self, *args):
        # Put arguments into a list, and pass them to list's __init__()
        super(Polynomial, self).__init__(list(args))
        
    def __repr__(self):
        text = ""
        
        pow = 0 # Counter to keep track of what power we are on
        for term in self:
            if pow == 0:
                # For the constant term, don't print out x^0
                text += str(term)
            else:
                text += "{0}x^{1}".format(term, pow)
            
            # Add plus sign after term if it's not the last term
            if pow + 1 < len(self):
                pow += 1
                text += " + "
            
        return text

In [4]:
x = Polynomial(1, 2, 3)
x

1 + 2x^1 + 3x^2

## Implementing Addition
Now we will move on to implementing addition between polynomials.

In [5]:
x + x

[1, 2, 3, 1, 2, 3]

### What happened above
As mentioned earlier, our `Polynomial` object inherits all of `list`'s methods. The addition operator for lists simply joins the lists together, but this is not what we want to happen here. So, we will override the `__add__()` method which controls how the `+` operator functions.

The algorithm for this is straightfoward--add up the polynomials term by term, which we will do by iterating through the terms of the polynomials. For example, on the first loop we will add the first terms together, on the second loop we will add the second terms together, and so on. As you can see, if we have a polynomial of degree 10 and another polynomial of, say, degree 3, we will need to loop through 10 times to fully add up the terms this way. In general, the number of loops we need to go through is equal to the length of the longest polynomial.

Now, you might see a problem with this solution, but we will not worry about it yet.

In [6]:
class Polynomial(list):
    def __init__(self, *args):
        # Put arguments into a list, and pass them to list's __init__()
        super(Polynomial, self).__init__(list(args))
        
    def __repr__(self):
        text = ""
        
        pow = 0 # Counter to keep track of what power we are on
        for term in self:
            if pow == 0:
                # For the constant term, don't print out x^0
                text += str(term)
            else:
                text += "{0}x^{1}".format(term, pow)
            
            # Add plus sign after term if it's not the last term
            if pow + 1 < len(self):
                pow += 1
                text += " + "
            
        return text
    
    def __add__(self, other):
        # The result of __add__() should be a new polynomial object
        new_poly = Polynomial()
        
        # Keep track of what term we are on
        i = 0
        
        # The number of times we need to loop is the length of the longest polynomial
        while i < max(len(self), len(other)):
            # Add polynomials term by term
            new_poly.append(self[i] + other[i])
            i += 1

        return new_poly

In [7]:
x = Polynomial(1, 2, 3)
x + x

2 + 4x^1 + 6x^2

### The Problem
Now our `__add__()` method works all fine and dandy for polynomials of the same length, but it does not work for polynomials of different lengths.

In [8]:
x = Polynomial(1, 2, 3)
y = Polynomial(1, 1, 1, 1)
x + y

IndexError: list index out of range

As seen above, our `__add__()` method loops through 4 times to add up all the terms. However, on the fourth iteration, it tries to access the non-existent fourth term of the first polynomial, causing an `IndexError`.

### The Solution
Again, the solution to this problem is to fall back on how we were taught to add up polynomials in algebra class. Implicitly, we assume any missing terms are just zero. We can implement this behavior in Python by overriding the `__getitem__()` method, which controls how items are retrieved when accessed by a key (in this case, a numeric index).

In [9]:
class Polynomial(list):
    def __init__(self, *args):
        # Put arguments into a list, and pass them to list's __init__()
        super(Polynomial, self).__init__(list(args))
        
    def __repr__(self):
        text = ""
        
        pow = 0 # Counter to keep track of what power we are on
        for term in self:
            if pow == 0:
                # For the constant term, don't print out x^0
                text += str(term)
            else:
                text += "{0}x^{1}".format(term, pow)
            
            # Add plus sign after term if it's not the last term
            if pow + 1 < len(self):
                pow += 1
                text += " + "
            
        return text
    
    def __add__(self, other):
        # The result of __add__() should be a new polynomial object
        new_poly = Polynomial()
        
        # Keep track of what term we are on
        i = 0
        
        # The number of times we need to loop is the length of the longest polynomial
        while i < max(len(self), len(other)):
            # Add polynomials term by term
            new_poly.append(self[i] + other[i])
            i += 1

        return new_poly
    
    def __getitem__(self, key):
        # The index being accessed exists --> return item as usual
        if key < len(self):
            return super(Polynomial, self).__getitem__(key)
        
        # The index being accessed is out of range --> return 0
        else:
            return 0

#### Testing our solution

In [10]:
x = Polynomial(1, 2, 3)
y = Polynomial(1, 1, 1, 1)
x + y

2 + 3x^1 + 4x^2 + 1x^3

## Subtraction
Now that we have implemented addition, we can implement subtraction. While we can implement `__sub__()` exactly like we implemented `__add__()`, we can craft a more concise, elegant solution by using `__add__()` in `__sub__()`. In order to subtract polynomials, all we have to do is flip the signs of the second polynomial, and add that to our first polynomial.

### Flipping Signs
By "flipping signs", I basically mean that whenever we have some polynomial defined like:
`x = Polynomial(1, 1, 1)`,

then we can flip the sign on each coefficient by simply typing `-x`.

Not surprisingly, the `-` sign when used as a "prefix" for a variable is also a method--`__neg__()`--we can override.

In [11]:
class Polynomial(list):
    def __init__(self, *args):
        # Put arguments into a list, and pass them to list's __init__()
        super(Polynomial, self).__init__(list(args))
        
    def __repr__(self):
        text = ""
        
        pow = 0 # Counter to keep track of what power we are on
        for term in self:
            if pow == 0:
                # For the constant term, don't print out x^0
                text += str(term)
            else:
                text += "{0}x^{1}".format(term, pow)
            
            # Add plus sign after term if it's not the last term
            if pow + 1 < len(self):
                pow += 1
                text += " + "
            
        return text
    
    def __add__(self, other):
        # The result of __add__() should be a new polynomial object
        new_poly = Polynomial()
        
        # Keep track of what term we are on
        i = 0
        
        # The number of times we need to loop is the length of the longest polynomial
        while i < max(len(self), len(other)):
            # Add polynomials term by term
            new_poly.append(self[i] + other[i])
            i += 1

        return new_poly
    
    def __getitem__(self, key):
        # The index being accessed exists --> return item as usual
        if key < len(self):
            return super(Polynomial, self).__getitem__(key)
        
        # The index being accessed is out of range --> return 0
        else:
            return 0
    
    def __neg__(self):
        new_poly = Polynomial()
        
        for term in self:
            new_poly.append(-term)
        
        return new_poly        
    
    def __sub__(self, other):
        return self.__add__(-other)

In [12]:
x = Polynomial(1, 1, 1, 1)
y = Polynomial(1, 1, 1, 2)
x - y

0 + 0x^1 + 0x^2 + -1x^3

## Exercise
Implementing polynomial multiplication and division has been left as an exercise for the reader.

### Other Exercises
1. Rewrite the `__repr__()` method so that zero terms don't appear, e.g. the last example simply outputs "-1x^3".
2. Create a method, say `eval()`, that allows you to evaluate the polynomial at some number.